# unzip the weather inmages

In [7]:
import zipfile
import os

# Specify the path to the zip file
zip_file_path = "dataset2_extracted/dataset2.zip"

# # Specify the directory where you want to extract the contents
# extract_to_directory = ""

# # Create the directory if it doesn't exist
# os.makedirs(extract_to_directory, exist_ok=True)

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all contents to the specified directory
    zip_ref.extractall()

print("Extraction completed successfully.")


Extraction completed successfully.


In [5]:
ls 

4drtyfjtfy-1.zip  bank.ipynb           test_feature.csv
Untitled.ipynb    dataset2_extracted/  train.csv
Untitled1.ipynb   test.csv             validation.csv


In [8]:
import numpy as np
import pandas as pd
import os
import random
import shutil
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
%matplotlib inline

In [9]:
import os

# Specify the directory containing the image files
directory = "dataset2"

# Initialize a list to store the names of image files
files = []

# Iterate through the files in the specified directory
for file in os.listdir(directory):
    # Check if the file is a JPEG image
    if file.endswith('.jpg'):
        # Append the file name to the list
        files.append(os.path.join(directory, file))

# Display the list of image files
print("List of image files:")
for file in files[:10]:
    print(file)


List of image files:
dataset2/sunrise125.jpg
dataset2/cloudy269.jpg
dataset2/rain205.jpg
dataset2/rain16.jpg
dataset2/shine118.jpg
dataset2/rain124.jpg
dataset2/sunrise206.jpg
dataset2/cloudy188.jpg
dataset2/sunrise80.jpg
dataset2/shine190.jpg


In [12]:
# Splits the images into 80% training, 10% validation, and 10% testing
split_1 = int(0.8 * len(files))
split_2 = int(0.9 * len(files))

train_filenames = files[:split_1]
val_filenames = files[split_1:split_2]
test_filenames = files[split_2:]

print("Training filenames:")
for file in train_filenames[:3]:
    print(file)

print("\nValidation filenames:")
for file in val_filenames[:3]:
    print(file)

print("\nTesting filenames:")
for file in test_filenames[:3]:
    print(file)


Training filenames:
dataset2/sunrise125.jpg
dataset2/cloudy269.jpg
dataset2/rain205.jpg

Validation filenames:
dataset2/sunrise218.jpg
dataset2/rain136.jpg
dataset2/sunrise272.jpg

Testing filenames:
dataset2/cloudy251.jpg
dataset2/cloudy80.jpg
dataset2/cloudy134.jpg


# create folders and load images files into those folders

In [13]:
import boto3
from PIL import Image
import os

# Initialize S3 client
s3 = boto3.client('s3')

# Define the bucket name
bucket_name = 'sagemaker-us-west-2-529165531209'

# Define the folder names
folders = ['train', 'validation', 'test']

# Loop through each folder
for folder in folders:
    # Create the folder in S3 bucket
    s3.put_object(Bucket=bucket_name, Key=(folder+'/'))

# Define the image size for resizing
image_size = (224, 224)

# Define the folders containing filenames
folders_filenames = {'train': train_filenames, 'validation': val_filenames, 'test': test_filenames}

# Loop through each folder and its filenames
for folder, filenames in folders_filenames.items():
    # Loop through each filename in the folder
    for filename in filenames:
        # Open the image file
        with Image.open(filename) as img:
            # Resize the image
            img_resized = img.resize(image_size)
            # Convert image to bytes
            img_bytes = img_resized.tobytes()
            # Define the S3 key (object key)
            s3_key = os.path.basename(filename)
            # Upload the resized image to S3 bucket in the corresponding folder
            s3.put_object(Bucket=bucket_name, Key=(folder+'/'+s3_key), Body=img_bytes)
            print(f"Uploaded {s3_key} to S3 bucket {bucket_name}/{folder} after resizing.")

print("Image resizing and upload completed successfully.")


Uploaded sunrise125.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded cloudy269.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded rain205.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded rain16.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded shine118.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded rain124.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded sunrise206.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded cloudy188.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded sunrise80.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded shine190.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded shine55.jpg to S3 bucket sagemaker-us-west-2-529165531209/train after resizing.
Uploaded

Create the metadata file(lst file). The file should be in the format of

image_index, label, image_name/image_path

Since this is a multilabel classification problem

image_index, label_1, label_2,..label_n, image_name/image_path

In [27]:
import boto3
import pandas as pd
from io import StringIO

# Initialize S3 client
s3 = boto3.client('s3')

# Define the bucket name
bucket_name = 'sagemaker-us-west-2-529165531209'

# Function to get labels based on file names
def get_label(file):
    if 'cloudy' in file:
        return 0
    elif 'rain' in file:
        return 1
    elif 'shine' in file:
        return 2
    else:
        return 3

# Function to create DataFrame from S3 folder
def create_dataframe_from_s3_folder(folder_name):
    df = pd.DataFrame()
    filenames = []
    labels = []
    
    # List objects in the S3 folder
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)
    for obj in response.get('Contents', []):
        filenames.append(obj['Key'])
        labels.append(get_label(obj['Key']))
    
    # Create DataFrame
    df['labels'] = labels
    df['s3_path'] = filenames
    df = df.reset_index().rename(columns={'index':'row_id'})
    df = pd.concat([df, pd.get_dummies(df['labels'], prefix='label')], axis=1).drop('labels', axis=1)
    new_columns = ['row_id', 'label_0', 'label_1', 'label_2', 'label_3', 's3_path']
    df = df.reindex(columns=new_columns)
    return df

# Create DataFrame for training set
train_df = create_dataframe_from_s3_folder('train/')


# Create DataFrame for validation set
validation_df = create_dataframe_from_s3_folder('validation/')



In [37]:
# train_df.fillna('False', inplace = True)
# train_df = train_df.replace(True,1)
# train_df = train_df.replace(False,0)
# train_df = train_df.replace('False',0)
# train_df = train_df.iloc[1:,:]
# train_df.head()

,row_id,label_0,label_1,label_2,label_3,s3_path
1,1,1,0,0,0,train/cloudy1.jpg
2,2,1,0,0,0,train/cloudy10.jpg
3,3,1,0,0,0,train/cloudy100.jpg
4,4,1,0,0,0,train/cloudy101.jpg
5,5,1,0,0,0,train/cloudy102.jpg


In [38]:
validation_df.fillna('False', inplace = True)
validation_df = validation_df.replace(True,1)
validation_df = validation_df.replace(False,0)
validation_df = validation_df.replace('False',0)
validation_df = validation_df.iloc[1:,:]
validation_df.head()

,row_id,label_0,label_1,label_2,label_3,s3_path
1,1,1,0,0,0,validation/cloudy103.jpg
2,2,1,0,0,0,validation/cloudy116.jpg
3,3,1,0,0,0,validation/cloudy129.jpg
4,4,1,0,0,0,validation/cloudy13.jpg
5,5,1,0,0,0,validation/cloudy138.jpg


In [39]:
validation_df.head()

,row_id,label_0,label_1,label_2,label_3,s3_path
1,1,1,0,0,0,validation/cloudy103.jpg
2,2,1,0,0,0,validation/cloudy116.jpg
3,3,1,0,0,0,validation/cloudy129.jpg
4,4,1,0,0,0,validation/cloudy13.jpg
5,5,1,0,0,0,validation/cloudy138.jpg


# load the files to the bucket

In [40]:
from io import StringIO
import boto3
import pandas as pd

# Initialize S3 client
s3 = boto3.client('s3')

# Define the bucket name
bucket_name = 'sagemaker-us-west-2-529165531209'

# Function to upload DataFrame to S3
def upload_dataframe_to_s3(dataframe, filename, bucket_name):
    csv_buffer = StringIO()
    dataframe.to_csv(csv_buffer, sep='\t', index=False, header=False)
    response = s3.put_object(Bucket=bucket_name, Key=filename, Body=csv_buffer.getvalue())
    print(f"Uploaded DataFrame to S3 bucket {bucket_name}/{filename}")

# Load train_df to S3
upload_dataframe_to_s3(train_df, 'train.lst', bucket_name)

# Load validation_df to S3
upload_dataframe_to_s3(validation_df, 'validation.lst', bucket_name)


Uploaded DataFrame to S3 bucket sagemaker-us-west-2-529165531209/train.lst
Uploaded DataFrame to S3 bucket sagemaker-us-west-2-529165531209/validation.lst


In [41]:
train_df.to_csv('train.lst', sep='\t', index=False, header=False)
validation_df.to_csv('validation.lst', sep='\t', index=False, header=False)